네이버에서 ETF 종목 추출

In [ ]:
# import sys
import csv
import requests
import json
import pandas as pd
import datetime

today = datetime.date.today().strftime("%y%m%d")
filename = today + "_etf_code.csv"
f = open(filename, "w", encoding="euc-kr", newline="")

url = 'https://finance.naver.com/api/sise/etfItemList.nhn'
json_data = json.loads(requests.get(url).text)
df = pd.json_normalize(json_data['result']['etfItemList'])

df.to_csv(f)
f.close()
# df.to_excel('etf_list_20251030.xlsx')

print("save")


save


기간별 수익률

In [ ]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json

def fn_etf_data(code, name):
    url = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A"+code+"&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701"
    #print(url)
    
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')    
    #print(res.text)
    #sys.exit()
    script = soup.find_all("script")[7].text
    #print(script)
    
    url_su = "http://comp.fnguide.com/SVO2/xml/Snapshot_all/{}.xml".format(code)
    
    res_su = requests.get(url_su)
    res_su.raise_for_status()
    soup_su = BeautifulSoup(res_su.text, 'xml')  
    #print(soup_su)
    
    try:        
        #s_name = soup.find("price").find("close_val").text.replace(u'\xa0','')
        #s_name = " "
        #s_name = soup.shareholders_type.get_text()
        #/html/body/div[1]/xml/upjong_B/record[1]/item_nm/text()

        s_code = "e=" + code
        s_name = name

        s_price = soup_su.find("close_val").text
        
        s_max = soup_su.find("high52week").text
        s_min = soup_su.find("low52week").text
        
        #기간별 수익률
        s_1m = soup_su.find("change_1month").text
        s_3m = soup_su.find("change_3month").text
        s_6m = soup_su.find("change_6month").text
        s_1y = soup_su.find("change_12month").text        
        s_3y = soup_su.find("change_36month").text

        s_mksum = soup_su.find("market_sum").text
                
        ####################################################

        # etf1FundInfoData 배열 부분만 추출
        pattern = re.compile(r"var\s+etf1FundInfoData\s*=\s*(\[.*?\]|\{.*?\});", re.DOTALL)
        match = pattern.search(script)
        #print(script)
        #print(match)

        if match:
            array_text = match.group(1)

            # 3. HTML 태그(<span ...>) 제거
            clean_text = re.sub(r"<.*?>", "", array_text)

            # 4. JSON으로 변환
            data = json.loads(clean_text)

            # "val01"을 키로, "val02"를 값으로 딕셔너리화
            # etf_dict = {}
            # for item in data2:
            #     etf_dict[item["val01"]] = item["val02"]
            #     etf_dict[item["val03"]] = item["val04"]

            # 배열일 수도 있고 객체일 수도 있음
            if isinstance(data, list):
                etf_dict = {item["val01"]: item["val02"] for item in data}
                for item in data:
                    etf_dict[item["val03"]] = item["val04"]
            elif isinstance(data, dict):
                etf_dict = {
                    data["val01"]: data["val02"],
                    data["val03"]: data["val04"]
                }

            #print(etf_dict)

            s_fee = etf_dict.get("총보수")
            s_setupday = etf_dict.get("설정일")
            s_setupamt = etf_dict.get("설정액/순자산").split("/")
              
        #분배금
        s_b1 = soup.find("div", attrs={"id":"etfDivInfo1"}).find_all("td")[0].text
        s_b2 = soup.find("div", attrs={"id":"etfDivInfo1"}).find_all("td")[1].text
        s_b3 = soup.find("div", attrs={"id":"etfDivInfo1"}).find_all("td")[2].text
        s_b4 = soup.find("div", attrs={"id":"etfDivInfo1"}).find_all("td")[3].text
        #//*[@id="etfDivInfo1"]/table/tbody/tr[1]/td[1]
        # print(s_b1)
        
        #펀드현황 및 분배금
        
        r_data = [s_name, s_code, s_price, s_1m, s_3m, s_6m, s_1y, s_3y, s_max, s_min, s_mksum, s_fee, s_setupday, s_setupamt[0], s_setupamt[1], s_b1, s_b2, s_b3, s_b4]
        #r_data = [s_name, s_code, s_price, s_1m, s_3m, s_6m, s_1y, s_3y, s_max, s_min]

        #print(r_data)
        return r_data
    
    except:
        #s_name = ""
        s_b1 = ""
        s_b2 = ""
        s_b3 = ""
        s_b4 = ""
        
        s_1m = ""
        s_3m = ""
        s_6m = ""
        s_1y = ""
        s_3y = ""
        s_max = ""
        s_min = ""
        
        print(s_name+" error")        


    #print("---------------------------------------")

In [ ]:
import datetime

#today = datetime.date.today().strftime("%y%m%d")
today = "251208"

filename = today + "_etf_period.csv"
f = open(filename, "w", encoding="euc-kr", newline="")
writer = csv.writer(f)

title = "종목명,종목코드,현재가,1개월,3개월,6개월,1년,3년,최고가,최저가,시가총액,총보수,설정일,설정액,순자산,최근분배금,지급일자,연분배횟수,배당수익률".split(",")

#print(type(title))
writer.writerow(title)


# 종목
#codes = ['005930', '005380', '012330', '105560', '033780', '000810', '001040', '294870', '237690', '086450', '230360', '183490', '030520', '089030', '095340', '000640', '073490', '010130', '035420', '035720']

#df1 = pd.read_csv('D:/정리 2020/작업/fn가이드/수집종목.csv', encoding="euc-kr")

file_etf = today + "_etf_code.csv"
df_etf = pd.read_csv(file_etf, encoding="euc-kr")
df_etf_select = df_etf.loc[(df_etf['marketSum'] > 300) & (df_etf['quant'] > 10000)]
codes = df_etf_select['itemcode']
names = df_etf_select['itemname']

#print(codes)
i = int()

for code, name in zip(codes, names):
    #print(code[2:])
    i = i + 1
    if i%100==0 : print(i)
    data = fn_etf_data(code, name)   
    if data :
        #print(code)
        #print(data)        
        writer.writerow(data)

print('save')
f.close()

http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A360750&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701
http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A069500&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701
http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A488770&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701
http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A459580&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701
http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A133690&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701
http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A379800&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701
http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A357870&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701
http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A102110&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701
http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A379810&cID=&MenuYn=Y&ReportG

In [ ]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json

code = "360750"
url = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A"+code+"&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701"
print(url)

res = requests.get(url)
res.raise_for_status()
soup = BeautifulSoup(res.text, 'lxml')    
#print(res.text)

script = soup.find_all("script")[7].text
#print(script)

# 2. etf1PriceData 배열 부분만 추출
pattern = re.compile(r"var\s+etf1PriceData\s*=\s*(\[.*?\]);", re.DOTALL)
match = pattern.search(script)

if match:
    array_text = match.group(1)

    # 3. HTML 태그(<span ...>) 제거
    clean_text = re.sub(r"<.*?>", "", array_text)

    # 4. JSON으로 변환
    data = json.loads(clean_text)

    # "val01"을 키로, "val02"를 값으로 딕셔너리화
    etf_map = {item["val01"]: item["val02"] for item in data}

# 2. etf1FundInfoData 배열 부분만 추출
pattern2 = re.compile(r"var\s+etf1FundInfoData\s*=\s*(\[.*?\]);", re.DOTALL)
match2 = pattern2.search(script)

if match2:
    array_text2 = match2.group(1)

    # 3. HTML 태그(<span ...>) 제거
    clean_text2 = re.sub(r"<.*?>", "", array_text2)

    # 4. JSON으로 변환
    data2 = json.loads(clean_text2)

    # "val01"을 키로, "val02"를 값으로 딕셔너리화
    etf_dict = {}
    for item in data2:
        etf_dict[item["val01"]] = item["val02"]
        etf_dict[item["val03"]] = item["val04"]




print(etf_map)
print(etf_dict)


http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A360750&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701
{'전일종가/전일대비(원)': '25,080/+0(0.00%)', '52주 최고가(원)': '25,080', '52주 최저가(원)': '17,636', '수익률(1M/3M/6M/1Y)(%)': '+2.98/+12.14/+25.11/+18.19', '시가총액': '120,672', '발행주식수': '481,150,000', '외국인보유율': '0.00', '거래량(20일 평균,주)': '4,323,570', '거래대금(20일 평균,원)': '106,547,717,386'}
{'운용사': '미래에셋자산운용', '기준가(원)': '25009.46', '매니저': '공동', '총보수': '0.0068', '판매사': '', '운용보수': '0.0002', '설정일': '2020/08/06', '판매보수': '0.0001', '존속기간': '5년4개월', '수탁보수': '0.005', '설정액/순자산': '48,090/120,270', '일반사무보수': '0.0015'}
